In [1]:
import geopandas as gpd
import pandas as pd
from shapely.ops import cascaded_union
import numpy as np
from shapely.geometry import Point, Polygon

import matplotlib.pyplot as plt
import psycopg2
import shapely.wkb as wkt
from geopandas import GeoSeries
from shapely import wkb
import pdb

In [6]:
ds = gpd.read_file(r'C:\Users\mmazzolini\OneDrive - Scientific Network South Tyrol\CCM2\wgs_84\SELECTED.shp')
ds_hierarchy=ds[['WSO1_ID','NEXTDOWNID']]

In [3]:
def find_upstream_poly(origin ,lista=[]):
    upper=ds[ds.NEXTDOWNID==origin]
    for i in upper.iterrows():
        lista.append(i[1].geometry)
        lista = find_upstream_poly(i[1].WSO1_ID,lista)
        
    return lista;  

In [4]:
conn = psycopg2.connect(host="10.8.244.31",
                   database="climate_data",
                   user="ado_admin",
                   password="oda347hydro",
                   port=5432)
cur = conn.cursor()

# get the metadata
query = f"""
        SELECT "id_station","geom" AS "point" FROM "hydrology"."metadata_disc" WHERE "region" = 'Emilia Romagna'
        """
meta = pd.read_sql_query(query, conn)

# close the connection when finished
cur.close()
conn.close()

In [5]:
meta

,id_station,point
0,ADO_DSC_ITH5_0001,0101000020E61000003160C9552C1E2540BDA8DDAF0274...
1,ADO_DSC_ITH5_0002,0101000020E61000001422E010AA8425406F2F698CD685...
2,ADO_DSC_ITH5_0003,0101000020E6100000ACE122F774FD2340BE326FD57590...
3,ADO_DSC_ITH5_0004,0101000020E6100000B03BDD79E2D926406CD097DEFE78...
4,ADO_DSC_ITH5_0005,0101000020E6100000E15D2EE23B692340B0E42A16BF87...
5,ADO_DSC_ITH5_0006,0101000020E6100000043A9336553727407495EEAEB371...
6,ADO_DSC_ITH5_0007,0101000020E6100000DB696B4430962640EF8E8CD5E681...
7,ADO_DSC_ITH5_0008,0101000020E6100000B745990D32B122404A7F2F85078D...


In [7]:
not_found_catchments=[]
path=r'C:\Users\mmazzolini\OneDrive - Scientific Network South Tyrol\CCM2\final_del/'
polygons=[]
for i in meta.iterrows():
    
    
    position=GeoSeries(wkb.loads(i[1].point, hex=True))
    
    for j in ds.iterrows():
        #poly=GeoSeries(wkb.loads(j[1].geometry, hex=True))
        if j[1].geometry.contains(position[0]):
            #print(j[1].WSO1_ID)
            origin=j[1].WSO1_ID
            
    del polygons[:]
    
    current=ds[ds.WSO1_ID==origin]
    
    polygons=find_upstream_poly(origin)
    
    if not polygons:
        not_found_catchments.append(i[1].id_station)
        print('cannot find catchment for:')
        print(i[1].id_station)

    else:
        for g in current.iterrows():
            polygons.append(g[1].geometry)
        catchment = gpd.GeoSeries(cascaded_union(polygons))
        d = {'id_station': [i[1].id_station], 'geometry': [catchment[0]]}
        df= gpd.GeoDataFrame(d).set_crs('epsg:4326')
        df.to_file((path+f"{i[1].id_station}.shp"))
        

    #pdb.set_trace()
    

cannot find catchment for:
ADO_DSC_ITH5_0008
